### `MultiNodeWeightedSampler` example
In this notebook, we will explore the usage of `MultiNodeWeightedSampler` in `torchdata.nodes`.

`MultiNodeWeightedSampler` allows us to sample with a probability from multiple datsets. We will make three datasets, and then see how does the composition of the output depends on the weights defined in the `MultiNodeWeightedSampler`.

In [1]:
from torchdata.nodes import Mapper, MultiNodeWeightedSampler, IterableWrapper, Loader
import collections

# defining a simple map_fn as a place holder example
def map_fn(item):
    return {"x":item}


def constant_stream(value: int):
  while True:
    yield value

# First, we create a dictionary of three datasets, with each dataset converted into BaseNode using the IterableWrapper
num_datasets = 3
datasets = {
    "ds0": IterableWrapper(constant_stream(0)),
    "ds1": IterableWrapper(constant_stream(1)),
    "ds2": IterableWrapper(constant_stream(2)),
}

# Next, we have to define weights for sampling from a particular dataset
weights = {"ds0": 0.5, "ds1": 0.25, "ds2": 0.25}

# Finally we instatiate the MultiNodeWeightedSampler to sample from our datasets
multi_node_sampler = MultiNodeWeightedSampler(datasets, weights)

# Since nodes are iterators, they need to be manually .reset() between epochs.
# We can wrap the root node in Loader to convert it to a more conventional Iterable.
loader = Loader(multi_node_sampler)

In [2]:
# Let's take a look at the output for 100k numbers, compute the fraction of each dataset in that batch
# and compare the batch composition with our given weights
n = 100000
it = iter(loader)
samples = [next(it) for _ in range(n)]
fractions = {k: v/len(samples) for k, v in collections.Counter(samples).items()}
print(f"fractions = {fractions}")
print(f"The original weights were = {weights}")

fractions = {0: 0.49791, 2: 0.25067, 1: 0.25142}
The original weights were = {'ds0': 0.5, 'ds1': 0.25, 'ds2': 0.25}
